In [1]:
from __future__ import division
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import copy
import random

N =  50                 # Number of nodes
rw = 700                 # Wealth value of rich
pw = 300                # Wealth value of poor
we_arr = [rw, pw]       # Array storing both the wealth values
pr = 0.5                # Probability of being rich
prob_arr = [pr, 1-pr]  # Array storing probability of wealth allotment values
M = 20                  # Number of rounds
co = 50                 # Unitary Cost of co-operation
be = 100                # Unitary Benefit of co-operation
re = 0.3                # Rewiring probability
S = 50                  # Number of iterations
pl = 0.3                # Initial probability of a link being formed between any two nodes

def inv_logit(y):
    inv_logit_out =(np.exp(y)/(1+np.exp(y)))
    return inv_logit_out

def gini_calc(x):       #IN: Wealth values across the population, OUT: Gini Coefficient
    sum2=0
    for i in range(len(x)):
        sum1=0
        for j in range(len(x)):
            sum1=sum1+np.abs(x[i]-x[j])
        sum2=sum2+sum1;
    
    gini=sum2/(2*len(x)**2*np.mean(x));
    return gini

node_arr = np.arange(N)
p0_ar = np.arange(0.55,1.05,0.05)

rc_ar = np.zeros([len(p0_ar),S])   # Array for storing no of cooperations by rich individuals
rd_ar = np.zeros([len(p0_ar),S])   # Array for storing no of defections by rich individuals
pd_ar = np.zeros([len(p0_ar),S])   # Array for storing no of defections by poor individuals
pc_ar = np.zeros([len(p0_ar),S])   # Array for storing no of cooperations by poor individuals
c_env_ar = np.zeros([len(p0_ar),S]) # Array storing no of cooperative environments 
d_env_ar = np.zeros([len(p0_ar),S])  # Array storing no of selfish environments 
eq_env_ar = np.zeros([len(p0_ar),S])  # Array storing no of neutral environments 
coop_ar = np.zeros([len(p0_ar),S])    # Array storing no of cooperators
def_ar = np.zeros([len(p0_ar),S])   # Array storing no of defectors 

flag = 0

for h1 in range(len(p0_ar)): 
    p0 = p0_ar[h1]
    for h in range(S):
        
        flag += 1
        rc = 0         # Flag for no of rich cooperators
        rd = 0         # Flag for no of rich defectors
        pd = 0         # Flag for no of poor defectors
        pcop = 0       # Flag for no of poor cooperators
        c_env = 0      # Flag for no of cooperative environments
        d_env = 0      # Flag for no of selfish environments
        eq_env = 0     # Flag for no of neutral environments
        f_c = 0       # Flag for no of cooperators
        f_d = 0        # Flag for no of defectors
        
        G = nx.erdos_renyi_graph(N, pl)
        A = nx.adjacency_matrix(G)
        No_of_rewiring = round((N*(N-1)/2)*re)
        Nre = int(No_of_rewiring)        
        init_wealth_array = np.random.choice(we_arr,N,p=prob_arr) 
        GINI = gini_calc(init_wealth_array)       
            
        for k in range(M):                 # Continuing the game for M rounds
            env_array = np.zeros(N)
            if (k==0):                    # First round  
                
                decision_array = np.zeros(N)
                for i in range(N):         # For all nodes in the network
                    nn = G.degree(i)
                    a = np.random.random()
                    p_in = 0.7
                    if (a <= p_in):
                        decision_array[i] = 1       # Node-i decides to be cooperative                    
                        init_wealth_array[i] = init_wealth_array[i] - co*nn   # Updatation of wealth of a cooperative node
                        for l in range(N):          # Updatation of wealth of its neighbouring nodes
                            if (i==l):
                                continue
                            else:
                                if (A[i,l] == 1):
                                    init_wealth_array[l] = init_wealth_array[l] + be
                                else:
                                    continue

                    else:
                        decision_array[i] = 0          # Node-i decides to be non-cooperative
                        continue   
                              
                
            else:                     # Decision making in subsequent rounds
                new_decision_array = np.zeros(N)
                new_env_array = np.zeros(N)
                for i in range(N):         # For all nodes in the network
                                                        
                    cop = 0                     # Initializing the no of cooperator neighbours
                    for j in range(N):
                        if (A[i,j] == 1):               # If a link exists between nodes i and j
                            if (decision_array[j] == 1):        # If link j cooperated in last round
                                cop += 1
                            elif (decision_array[j] == 0):      # If link j did not cooperate in last round
                                continue
                        else:
                            continue
                
                    sum_neighbor_wealth = 0
                    for j in range(N):			# obtaining total wealth of the neighbors
                        if (A[i,j] == 1):
                            sum_neighbor_wealth += init_wealth_array[j]
                        else:
                            continue
                 
                    if (G.degree(i)>0):                  # For postive number of nearest neighbours
                        av_neighbor_wealth = (sum_neighbor_wealth)/G.degree(i)
                        diff_wealth = av_neighbor_wealth - (init_wealth_array[i])
                        pc = p0 + (1-p0)*(np.tanh(0.001*diff_wealth)) - 0.10 
                                               
                        cop_rat = float(cop/G.degree(i)) 
                        
                        if (cop_rat > 0.5):       # If cooperation parameter is greater than 0.5 i.e cooperative environment
                            
                            c = np.random.random()
                            if (c < pc):
                                new_decision_array[i] = 1       # Node-i decides to be cooperative                          
                            else:
                                new_decision_array[i] = 0

                        elif (cop_rat < 0.5):          # Cooperation parameter is lesser than 0.5 i.e selfish environment
                            
                            c = np.random.random()
                            if (c < pc):                        
                                new_decision_array[i] = 0
                            else:
                                new_decision_array[i] = 1
                            
                        elif (cop_rat == 0.5):        # Neutral environment
                              
                              c = np.random.random()
                              if (c<=pc):
                                  new_decision_array[i] = 1
                              else:
                                  new_decision_array[i] = 0
                                    
                        if (new_decision_array[i]==1):
                            if (diff_wealth<0):
                                rc += 1
                            else:
                                pcop += 1
                        elif (new_decision_array[i]==0): 
                            if (diff_wealth>0):
                                pd += 1
                            else:
                                rd += 1
                                
                    else:
                        new_decision_array[i] = 0                 
                
                # Wealth update
                for i in range(N):
                    if (new_decision_array[i] == 1): 
                        init_wealth_array[i] = init_wealth_array[i] - co*(G.degree(i))   # Updatation of wealth of a cooperative node
                        for l in range(N):          	# Updatation of wealth of its neighbouring nodes
                            if (i==l):
                                continue
                            else:
                                if (A[i,l] == 1):
                                    init_wealth_array[l] = init_wealth_array[l] + be
                                else:
                                    continue
                    else:
                        continue       
                               
                
                
                decision_array = copy.copy(new_decision_array)        
                    
            # Rewiring step
                   
            temp_array = []                                                
            
            while (len(temp_array) < Nre) :            
                i = random.choice(node_arr)
                j = random.choice(node_arr)
                    
                if (i==j):
                    continue
                else:
                
                    if (i,j) in temp_array:
                        continue
                    elif (j,i) in temp_array:
                        continue
                    else:
                        temp_array.append((i,j))
                        temp = [i,j]
                                                             
                        if (A[i,j] == 1):                              # If a link exists between the selected pair
                            b = random.choice(temp)
                            
                            if (b == i):                               # If node-i is given the choice to break or not
                                if (decision_array[j] == 0):
                                    c = np.random.random()
                                    if (c < 0.7):
                                        G.remove_edge(i,j)                 # Breakage of link for node-i disagreeing
                                  
                                    else:
                                        continue                        # Link is kept intact
                                else:
                                    c = np.random.random()
                                    if (c < 0.87):
                                        continue
                                    else:
                                        G.remove_edge(i,j)
                            else:                                   # If node-j is given the choice to break or not	
                                if (decision_array[i] == 0):
                                    c = np.random.random()
                                    if (c < 0.7):                                    
                                        G.remove_edge(i,j)         # Breakage of link for node-j disagreeing
                                    else:
                                        continue			# Link is kept intact
                                else:
                                    c = np.random.random()
                                    if (c < 0.87):
                                        continue
                                    else:
                                        G.remove_edge(i,j)
                                                                
            
                        else:                 # If a link does not exist between the selected pair
                            if (decision_array[i] == 1 and decision_array[j] == 1):
                                c = np.random.random()
                                if (c < 0.93):
                                    G.add_edge(i,j)
                                else:
                                    continue
                            
                            if (decision_array[i] == 0 and decision_array[j] == 0):
                                c = np.random.random()
                                if (c < 0.80):
                                    continue
                                else:
                                    G.add_edge(i,j)
                        
                            if (decision_array[i] == 0 and decision_array[j] == 1):
                                c = np.random.random()
                                if (c < 0.70):
                                    continue
                                else:
                                    G.add_edge(i,j)
                                
                            if (decision_array[i] == 1 and decision_array[j] == 0):
                                c = np.random.random()
                                if (c < 0.70):
                                    continue
                                else:
                                    G.add_edge(i,j)
                                
            A = nx.adjacency_matrix(G)         # Adjacency matrix of the network
            for i in range(N):         # For all nodes in the network                                                   
                    cop = 0                     # Initializing the no of cooperator neighbours
                    for j in range(N):
                        if (A[i,j] == 1):               # If a link exists between nodes i and j
                            if (decision_array[j] == 1):        # If link j cooperated in last round
                                cop += 1
                            elif (decision_array[j] == 0):      # If link j did not cooperate in last round
                                continue
                        else:
                            continue
                    if (G.degree(i)>0):
                        cop_rat = float(cop/G.degree(i))                    
                        if (cop_rat > 0.5):       # If cooperation parameter is greater than 0.5 i.e cooperative environment
                            env_array[i] = 1
                        elif (cop_rat == 0.5):  # Neutral environment
                            env_array[i] = 2
                        elif(cop_rat < 0.5):    # If cooperation parameter is lesser than 0.5 i.e selfish environment
                            env_array[i] = 0
                    else:
                        env_array[i] = 0
                        
            c_env += np.count_nonzero(env_array == 1)
            d_env += np.count_nonzero(env_array == 0)
            eq_env += np.count_nonzero(env_array == 2)
            
        rc_ar[h1,h] = rc 
        pd_ar[h1,h] = pd 
        rd_ar[h1,h] = rd 
        pc_ar[h1,h] = pcop 
        c_env_ar[h1,h] = c_env
        d_env_ar[h1,h] = d_env
        eq_env_ar[h1,h] = eq_env
        coop_ar[h1,h] = rc_ar[h1,h] + pc_ar[h1,h]
        def_ar[h1,h] = pd_ar[h1,h] + rd_ar[h1,h]
        
        yo = (flag*100)/(len(p0_ar)*S)
        print('Job completion percentage: %f' %yo)
        
np.savez('strat_environ_vs_p0', rc_ar=rc_ar, pd_ar=pd_ar, rd_ar=rd_ar, pc_ar=pc_ar, c_env_ar=c_env_ar, d_env_ar=d_env_ar, eq_env_ar=eq_env_ar, coop_ar=coop_ar, def_ar=def_ar)

Job completion percentage: 0.200000
Job completion percentage: 0.400000
Job completion percentage: 0.600000
Job completion percentage: 0.800000
Job completion percentage: 1.000000
Job completion percentage: 1.200000
Job completion percentage: 1.400000
Job completion percentage: 1.600000
Job completion percentage: 1.800000
Job completion percentage: 2.000000
Job completion percentage: 2.200000
Job completion percentage: 2.400000
Job completion percentage: 2.600000
Job completion percentage: 2.800000
Job completion percentage: 3.000000
Job completion percentage: 3.200000
Job completion percentage: 3.400000
Job completion percentage: 3.600000
Job completion percentage: 3.800000
Job completion percentage: 4.000000
Job completion percentage: 4.200000
Job completion percentage: 4.400000
Job completion percentage: 4.600000
Job completion percentage: 4.800000
Job completion percentage: 5.000000
Job completion percentage: 5.200000
Job completion percentage: 5.400000
Job completion percentage: 5

Job completion percentage: 44.800000
Job completion percentage: 45.000000
Job completion percentage: 45.200000
Job completion percentage: 45.400000
Job completion percentage: 45.600000
Job completion percentage: 45.800000
Job completion percentage: 46.000000
Job completion percentage: 46.200000
Job completion percentage: 46.400000
Job completion percentage: 46.600000
Job completion percentage: 46.800000
Job completion percentage: 47.000000
Job completion percentage: 47.200000
Job completion percentage: 47.400000
Job completion percentage: 47.600000
Job completion percentage: 47.800000
Job completion percentage: 48.000000
Job completion percentage: 48.200000
Job completion percentage: 48.400000
Job completion percentage: 48.600000
Job completion percentage: 48.800000
Job completion percentage: 49.000000
Job completion percentage: 49.200000
Job completion percentage: 49.400000
Job completion percentage: 49.600000
Job completion percentage: 49.800000
Job completion percentage: 50.000000
J

Job completion percentage: 89.200000
Job completion percentage: 89.400000
Job completion percentage: 89.600000
Job completion percentage: 89.800000
Job completion percentage: 90.000000
Job completion percentage: 90.200000
Job completion percentage: 90.400000
Job completion percentage: 90.600000
Job completion percentage: 90.800000
Job completion percentage: 91.000000
Job completion percentage: 91.200000
Job completion percentage: 91.400000
Job completion percentage: 91.600000
Job completion percentage: 91.800000
Job completion percentage: 92.000000
Job completion percentage: 92.200000
Job completion percentage: 92.400000
Job completion percentage: 92.600000
Job completion percentage: 92.800000
Job completion percentage: 93.000000
Job completion percentage: 93.200000
Job completion percentage: 93.400000
Job completion percentage: 93.600000
Job completion percentage: 93.800000
Job completion percentage: 94.000000
Job completion percentage: 94.200000
Job completion percentage: 94.400000
J